In [ ]:

import pyvo as vo
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.table import Table
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from photutils.utils import calc_total_error
import pandas as pd
from scipy.spatial import KDTree
import json


from scipy.optimize import curve_fit
from photutils.detection import DAOStarFinder
from astropy.stats import mad_std, sigma_clipped_stats
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import math

In [ ]:
%matplotlib inline

# running over all galaxies

## Best version

In [ ]:
####Defining the constants
# defining a function to calculate the distances between two sources.
def dist(p1, p2):
   return np.sqrt( (p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )


#defining a function that creates a circular mask around each source so that if something overlaps with it, that overlapping part is not included in the aperture photometry
def create_circular_mask(h,w,center,radius): # I did not end up needing this code.
   Y, X = np.ogrid[:h, :w] # creating an open (more memory efficient) coordinate grid of the image
   dist_from_center = np.sqrt((X-center[0])**2+ (Y-center[1])**2)
   mask = dist_from_center <= radius # so that everything inside the radius receives a mask
   return mask

#defining a function for the overlap area so that I can scale the overlap counts with it.
def overlap_area(radius, distance):
    Area = ((2 * radius**2) * np.arccos((distance / (2 * radius)))) - ( .5 * distance * np.sqrt( (4 * radius**2) - distance**2))
    return Area

# defining a function to convert fluxes to luminosity.  L=4πr2f.
def to_lum(r, flux, net_flx_err):
    luminosity = 4 * math.pi * r * 2 * flux
    luminosity_unc = 4 * math.pi * r**2 * net_flx_err
    return luminosity, luminosity_unc



# define a mapping of the bands into labels to make it easier
band_labels = {'w1': 'W1', 'w2': 'W2', 'w3': 'W3', 'w4': 'W4'}
flux_zmfd = {'w1': 309.54 ,'w2': 171.787,'w3': 31.674,'w4': 8.363} # check if these worked by looking at the band 4 code above
instr_zpmag = {'w1': 20.73,'w2': 19.56,'w3': 17.6 ,'w4':12.98 }
wavelengths = {'w1': 3.4 ,'w2': 4.6,'w3': 12,'w4': 22}


#define function to get flux density per unit frequency (energy units)
def flux_dens(net_flx, net_flx_err, wavelength):
   flux_density = (net_flx * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   flux_density_unc = (net_flx_err * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   return flux_density, flux_density_unc


#import huge csv and grab the name and ra and dec needed for each galaxy.
targetgals = pd.read_csv('../Data/inWISE.csv') # this should not be the one for all 120 and should rather be for the 74 of them.
#print(targetgals[0:20])
huge = pd.read_csv('../Data/Hugefiles/Source_Flux_All_Modified_5.csv')
columns = ['RA','Dec','Gname_Modified','Gname_Homogenized', 'ObsID', 'EXPOSURE', 'NET_LUM_APER_0.3-8.0', 'Hard_Flux_Color' , 'Soft_Flux_Color', 'Galactic_Distance']
g_huge = huge[columns]
#display(g_huge.head())


#group the x-ray sources for this galaxy. locate through merging
df1 = targetgals
df2 = g_huge


merged_data = pd.merge(df1, df2, left_on='source_id', right_on = 'Gname_Homogenized', how='inner')
columns1 = ['RA','Dec','Gname_Homogenized', 'ObsID', 'EXPOSURE', 'NET_LUM_APER_0.3-8.0', 'Hard_Flux_Color' , 'Soft_Flux_Color', 'Galactic_Distance']
Xray_sources = merged_data[columns1]

#group by galaxy name and the longest exposure time.
longest_exposure_obs = Xray_sources.loc[Xray_sources.groupby('Gname_Homogenized')['EXPOSURE'].idxmax()]

# aggregate all the sources associated with the obsid with the longest exposure time
aggregated_sources = Xray_sources[Xray_sources['ObsID'].isin(longest_exposure_obs['ObsID'])]

#create a list of all the names needed
galaxy_names = targetgals['source_id'].unique()

galaxy_sources = {}
grouped_sources = aggregated_sources.groupby('Gname_Homogenized')
'''print("\nGrouped sources:")
for group_name, group in grouped_sources:
   print(f"\nGroup: {group_name}")
   print(group)'''
#get all of the ra and dec sources for the galaxy in question
for group_name, group in grouped_sources:
   galaxy_sources[group_name] = {'ra' : group['RA'].values, 'dec' : group['Dec'].values, 'ObsID': group['ObsID'].values, 
                                 'NET_LUM_APER_0.3-8.0': group['NET_LUM_APER_0.3-8.0'].values,  'Hard_Flux_Color': group['Hard_Flux_Color'].values , 'Soft_Flux_Color': group['Soft_Flux_Color'].values, 'Galactic_Distance': group['Galactic_Distance'].values }
  
rows = []
#create subset for testing:

targetgals_subset = targetgals.iloc[0:5]

# Lookup and define a service for ALLWISE Atlas images
allwise_service = vo.dal.SIAService("https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?")

#loop through the galaxies
#print("\nAligned target galaxies and grouped sources:")
for galaxy in targetgals.itertuples():
    galaxy_name = galaxy.source_id
    #print(galaxy_name)
    #group = aligned_aggregatedsources.get_group(galaxy_name)
  
    # Print galaxy information
    #print(f"\nGalaxy: {galaxy_name}")
    #print(group)

    #define coordinates
    ra1 = galaxy.ra_x
    #print (ra1)
    dec1 = galaxy.dec_x
    pos = SkyCoord(ra=ra1, dec=dec1, unit= 'deg')
    #search the service for images covering within 1 arcsecond of the star. make this bigger if needed
    im_table = allwise_service.search(pos=pos, size= 1*u.arcsec)
    #im_table
    im_table.to_table().colnames
    #print(im_table)
    # get the Ra and dec values necessary for the kdtree and rest of the code
    if galaxy_name in galaxy_sources:
       ra = galaxy_sources[galaxy_name]['ra']
       dec = galaxy_sources[galaxy_name]['dec']
       obsid = galaxy_sources[galaxy_name]['ObsID']
       NET_LUM_APER_all = galaxy_sources[galaxy_name]['NET_LUM_APER_0.3-8.0']
       Hard_Flux_Color = galaxy_sources[galaxy_name]['Hard_Flux_Color']
       Soft_Flux_Color = galaxy_sources[galaxy_name]['Soft_Flux_Color']
       Galactic_Distance = galaxy_sources[galaxy_name]['Galactic_Distance']
       print(f"Galaxy: {galaxy_name}")
       #print("RA values:", ra)
       #print("Number of RA values:", len(ra))
    else:
       print(f"No sources found for galaxy: {galaxy_name}")
       continue # skip to the next galaxy if no sources were found
          
   ##running the for loop over every image and doing aperture photometry on each one
   #currently outputs as w4,w1,w2,w3 when querying the images. so index is 0.1.2.3 i want the index to be 0.3.2.1

    for i in [0, 3, 2, 1]:  # index is different for every single image, going to keep this anyways
        band_id = im_table[i]["sia_bp_id"].lower()  # Get band ID in lowercase
        if band_id in band_labels:
            #print(f'Band {band_labels[band_id]}: ')
            data_url = im_table[i].getdataurl()
            #Download the image and open it in Astropy
            fname = download_file(data_url, cache=True)
            image1= fits.open(fname)
            image_data= image1[0].data
            #print(data)
            #print(data_url)
            wcs = WCS(image1[0].header)
            #cuting out the image of the galaxy apart from the rest of the background.
            cutout = Cutout2D(image_data, pos, (437,437), wcs=wcs)
            wcs = cutout.wcs
            cutout_data = cutout.data
            #print(cutout_data)
            positions = wcs.world_to_pixel_values(ra, dec)
            positions = np.array(list(zip(positions[0], positions[1])))

            #define the distance threshold for the KDTree grouping (in pixels). 5 usually works best. any larger and the average of their position goes wonky
            distance_threshold = 5

            #build the KDTree for efficient grouping
            tree = KDTree(positions)

            #query the KDTree to find points within the defined radius of dist threshold and group them together
            groups = tree.query_ball_tree(tree, r=distance_threshold)
            # print(groups)
            # consolidating the groups. 'unique_groups' and 'seen': These are used to ensure that each group is processed only once.
            unique_groups = []
            seen = set()
            for group in groups:
                group = tuple(sorted(group))
                if group not in seen:
                    seen.add(group)
                    unique_groups.append(group)
                 # print(unique_groups)
            # for each unique group, the average postion of the detections is calulated so that only one source detection is used for aperture photometry instead of a bunch of the same sources being used.
             #represents the consolidated postion of potentially multiple detections of one source.
            grouped_positions = [positions[list(group)].mean(axis=0) for group in unique_groups]
            #print(grouped_positions)

            #print("Grouped positions for galaxy", galaxy_name, ":", grouped_positions)
            #define the Region(s) Of Interest (center x, center y, radius)
            ROI = [ ((x, y) , 9, 16, 23) for x,y in grouped_positions ] # (x, y, radius around target, inner r, outer r)   36.3636, 50.90909) may need to mke the radius bigger when goruping?
          
                # initialize valid rows plotting for the current image iteration
            valid_rows = []
           
           #now inputting the aperture photometry part
           # check for overlap and perform aperture photometry
            for i, ((x, y), r, annulus_inner, annulus_outer) in  enumerate(ROI):
                overlap_dict = []
                overlap = False # initialize overlap flag (A boolean flag overlap is set to False for each source to track if it overlaps with any other source. becomes true if they do overlap 
                acc_overlap = False # initialize the acceptable overlap flag.  false if there is no overlap, true if there is overlap and it is acceptable 
               
                for j, ((x2, y2), r2, annulus_inner2, annulus_outer2) in  enumerate(ROI): # apparently you can run a for loop over 2 objects by putting the second one inside the first. it iterates over every source again to then see if it overlaps at all with another source.
                    if i != j: # ensures that a source is not compared to itself! wow
                        #print(f'{x}, {y} / {x2}, {y2}')
                        #print(f"Checking positions: ({x}, {y}) and ({x2, {y2}})")
                        distance = dist((x, y) , (x2, y2))
                        #print(f"Distance: {distance}, Radii Sum: {r + r2}")
                        #print('dsitance', distance)
                        #print('Distance', distance)
                        #print('r1', r)
                        if distance < r + r2:  # if the distance is less than the size of the two radii added together, then they are overlapping.
                            #print(distance)
                            #print('yesif')
                            overlap_percent = (r + r2 - distance)/(r+r2)  # the amount they are overlapping divided by the total size of radii distance
                            #print('overlap perc', overlap_percent)
                            if overlap_percent > .5:
                                overlap = True # this way, if they overlap by more than 50% then they will not be usable because less than 50% of the flux extractable area can be seen.
                                #print('overlap is unacceptable: ', overlap)
                                overlap_aperture = np.nan
                                overlap_photo_table = np.nan
                                overlap_counts = np.nan
                                overlap_error = np.nan
                              
                            elif overlap_percent <= .5:
                                acc_overlap = True
                                #print('acceptable or no overlap: ', acc_overlap)
                                #Handle overlaps that are acceptable (less than the threshold, but still overlapping by a small percent)
                                overlap_aperture = CircularAperture((x2, y2), r2)
                                overlapping_area = overlap_area(r,distance) # using the function to define the overlapping area for the location of overlap
                                overlap_photo_table = aperture_photometry(cutout_data, overlap_aperture)
                                total_area = math.pi * r**2
                                overlap_counts = overlap_photo_table['aperture_sum'][0] * ( overlapping_area/total_area ) # scaling the counts by the overlapping area
                                overlap_error = np.sqrt(overlap_counts)
                                overlap_dict.append({'Position': ({x}, {y}),  'overlapping counts': overlap_counts, 'overlap_error': overlap_error})
                                #print('were here')
                                #print('Overlap counts', overlap_counts)
                                #print('overlapdict', overlap_dict)

                        else:
                            #print(' Prob skips distance if statement')
                            #print('not overlapping at all')
                            overlap_percent = np.nan
                            overlap_aperture = np.nan
                            overlap_photo_table = np.nan
                            overlap_counts = np.nan
                            overlap_error = np.nan
                       
                #print(acc_overlap)  
                if acc_overlap:
                    overlap_counts = 0
                    overlap_error = 0
                    # for a source, if it overlaps with more than one other source, add all of the counts for those overlapping regions
                    for row in overlap_dict:
                        if not np.isnan(row['overlapping counts']) and not np.isnan(row['overlap_error']):
                            overlap_counts += row['overlapping counts'] # now add in quadriture for the propagation of error for sources like this
                            overlap_error += row['overlap_error']**2
                    overlap_error= np.sqrt(overlap_error)
                    #print('Overlapping COUNTS HERE', overlap_counts)
                    # For the Target objects in the little aperture circle define their target apertures
                    target_aperture = CircularAperture((x,y),r,)
                   
                    #perform aperture photometry on target
                    target_photo_table = aperture_photometry(cutout_data, target_aperture)
                    target_counts = target_photo_table['aperture_sum'][0]

                    target_counts -= overlap_counts
                    # so that i dont take the log or sqrt of a negative number or zero and get an error
                    if target_counts > 0: #
                        target_error= np.sqrt(target_counts)
                        #propagated error of overlap error
                        target_overlap_counts_err = np.sqrt(target_error**2 + overlap_error**2)
                        #print(target_counts)
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to magnitude
                            target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux
                          
                            #propagation of uncertainty of flux conversion
                            Mcal_error = (2.5*target_overlap_counts_err) / (target_counts * np.log(10))
                            target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)

                    else:# to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        target_error = np.nan
                        target_overlap_counts_err = np.nan
                        target_flux = np.nan
                        target_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        target_counts = np.nan

                    # continuing on with the photometry under the "if acc_overlap"

                    #calculate area of target aperutue
                    target_area = target_aperture.area

                    # For the Background Annuli of outside of the Target
                    #define the background annulus for the target
                    annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)

                    #perform aperture photometry on annuli
                    annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                    annulus_counts = annulus_photo_table['aperture_sum'][0]
              
                    if annulus_counts > 0:
                        overlapannulus_error = np.sqrt(annulus_counts) # the error of the annulus for sources that overlap
                        # to avoid taking the log of zero or negative value
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                            annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                            #propagation of uncertainty of flux conversion
                            Mcal_error_ann = (2.5 * overlapannulus_error) / (annulus_counts * np.log(10))
                            annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)

                    else: 
                        annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        overlapannulus_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        annulus_counts = np.nan
                          
                    #calculate area of annulus
                    annulus_area = annulus_aperture.area

                    # do the calculations for including a Background aperture
              
                    #Calculating the net flux:
                    #calculate the mean background per pixel
                    bg_perpixel = annulus_flux/annulus_area
                    bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!

                    #calculate the total background in the target aperture
                    tot_bg = bg_perpixel * target_area
                    tot_bg_err = bg_perpixel_err * target_area ##propagation of error!

                    #Subtract background from the target flux
                    net_flx = target_flux - tot_bg
                    net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!
              
                    #flag the sources that overlap
                    rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i],
                                 'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [], 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner,
                           'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,'Flux Uncertainty': net_flx_err, 'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux',
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux, 'Acceptable overlapping?': 'Yes' if acc_overlap == True else 'Not acceptable or no overlap at all', 'Unacceptable overlapping?': 'Yes' if overlap == True else 'Acceptable or no overlap at all',
                               'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                    # Append valid results to valid_rows
                    valid_rows.append({
                       'band_id': {band_labels[band_id]},  'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i], 'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [],
                       'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                       'Annulus_Inner_Radius': annulus_inner,
                       'Annulus_Outer_Radius': annulus_outer,
                       'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux', 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                       'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [],  'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                  
                else: #perform all the normal aperture photometry stuff for those that do not overlap in any way.
            
                   # For the Target objects in the little aperture circle define their target apertures
                    target_aperture = CircularAperture((x,y),r,)
              
                    #perform aperture photometry on target
                    target_photo_table = aperture_photometry(cutout_data, target_aperture)
                    target_counts = target_photo_table['aperture_sum'][0]

                    if target_counts > 0:   # avoid taking the log of zero or negative value
                        target_error = np.sqrt(target_counts)
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to magnitude
                            target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                            #propagation of uncertainty of flux conversion
                            Mcal_error = (2.5*target_error) / (target_counts * np.log(10))
                            target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)

                    else:
                        target_error = np.nan
                        target_flux = np.nan
                        target_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        target_counts = np.nan

                    #calculate area of target aperutue
                    target_area = target_aperture.area

                    # For the Background Annuli of outside of the Target
                    #define the background annulus for the target
                    annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)

                    #perform aperture photometry on annuli
                    annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                    annulus_counts = annulus_photo_table['aperture_sum'][0]
                    # the error of the annulus for sources that overlap
                  
                    if annulus_counts > 0:
                        annulus_error = np.sqrt(annulus_counts)
                        # avoid taking the log of zero or negative value
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                            annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux


                            #propagation of uncertainty of flux conversion
                            Mcal_error_ann = (2.5 * annulus_error) / (annulus_counts * np.log(10))
                            annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)
                    else:
                        annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        annulus_error = np.nan
                        annulus_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        annulus_counts = np.nan
                          
                    #calculate area of annulus
                    annulus_area = annulus_aperture.area

                    # do the calculations for including a Background aperture
              
                    #Calculating the net flux:
                    #calculate the mean background per pixel
                    bg_perpixel = annulus_flux/annulus_area
                    bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!

                    #calculate the total background in the target aperture
                    tot_bg = bg_perpixel * target_area
                    tot_bg_err = bg_perpixel_err * target_area ##propagation of error!

                    #Subtract background from the target flux
                    net_flx = target_flux - tot_bg
                    net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!

                    if overlap:
                        overlap_counts = np.nan
                        overlap_error = np.nan
                        target_counts = np.nan

                    #   Append the result as a dictionary to the list named 'rows'
                    rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i], 
                                 'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [],
                                 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner, 'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,
                                   'Flux Uncertainty': net_flx_err,  'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux',
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux, 'Acceptable overlapping?': 'Yes' if acc_overlap == True else 'Not acceptable or no overlap at all','Unacceptable overlapping?': 'Yes' if overlap == True else 'Acceptable or no overlap at all',
                               'Annulus Counts': annulus_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [],   'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id],'Flux Density': [], 'Flux Density Uncertainty' : [] })  #will prolly have to change the name of the Flux here!!!
                        #"Low Flux" means that they either have zero flux or negative flux and so they are excluded from the plotting
              
                    # Append valid results to valid_rows
                    valid_rows.append({
                       'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i],'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [],
                       'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                       'Annulus_Inner_Radius': annulus_inner,
                       'Annulus_Outer_Radius': annulus_outer,
                       'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,
                       'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux',
                       'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr, 'Exists?': 'No', 'Point Source Position' : [], 'Offset in Arcseconds': [],
                       'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })

            #Source detection code
            mean, median, std = sigma_clipped_stats(cutout_data, sigma=3.0) 
            #print((mean, median, std))

            # subtract the background and find FWHM of sources at a certain threshold
            #started at fwhm= 3 and threshold = 5
            daofind = DAOStarFinder(fwhm=8, threshold=1*std) # find the stars in the image that have FWHMs of around 3 pixels and have peaks approximately 5-sigma above the background.
            sources = daofind(cutout_data - median) 
            #print(type(sources))
            # will likely run into iissues in the code below
            for col in sources.colnames: 
                if col not in ('id', 'npix'):
                    sources[col].info.format = '%.2f'  # for consistent table output
                        # sources.pprint(max_width=3000) 


                        #likely the flux labeled in this is not converted!
          
            # plot the image with marked locations of all the sources it detected.
            detected_positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
            apertures = CircularAperture(detected_positions, r=2)

            # Plotting for current image
            # Filter valid rows
            valid_rows_filtered = [row for row in valid_rows if row['Flag'] == 'Valid']
          
            # Was there a point source there?
            pixelsinarc = 0.0003819444391411 * 3600 ## 0.0003819444391411 is the number found in the header of the image for the scale of pixels in degrees for the fits image.
            detectedpos_all = []
            #rows = [row for row in rows if row['Flag'] == 'Valid']
            for row in valid_rows_filtered:
                apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
                for detected_position in detected_positions:
                    detected_x, detected_y = detected_position
                    distance = dist((row['X'], row['Y']), (detected_x, detected_y) ) #distance from the center of the xray source to the center of the detected source
                    if distance <= row['Radius']:
                        row['Exists?'] = 'Point Source Detected'
                        row['Point Source Position'].append((detected_x, detected_y))
                        dist_in_arc = distance * pixelsinarc
                        row['Offset in Arcseconds'].append((dist_in_arc))
                        detectedpos_all.append(detected_position)
            # used for creating apertures for sources within the radius of the X-ray source apertures.
            Yes = []
            YesRadius= 5
            for row in valid_rows_filtered:
                        apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
                        for detected_position in detected_positions:
                            detected_x, detected_y = detected_position
                            distance = dist((row['X'], row['Y']), (detected_x, detected_y) )
                            if distance <= YesRadius:
                                row['Exists?'] = 'Yes!!'
                                Yes.append((row['X'], row['Y']))
                                #else: # this does not seem to do what i want it to do
                               #row['Exists?'] = 'Detected but does not exist'
                               
            #doing flux density
            for row in valid_rows_filtered:
                net_flx = row['Net Flux (Jy)']
                net_flx_err = row['Flux Uncertainty']
                wavelength = row['Wavelength']
                #print(net_flx)
                flux_density, flux_density_unc = flux_dens(net_flx, net_flx_err, wavelength)
                row['Flux Density'].append(flux_density)
                row['Flux Density Uncertainty'].append(flux_density_unc)
                  
            # Update rows with valid_rows_filtered information
            for valid_row in valid_rows_filtered:
                for row in rows:
                    if row['band_id'] == valid_row['band_id'] and row['Region'] == valid_row['Region']:
                        row.update(valid_row)

            # Converting to luminosity  L=4πr2f. where r = distance to the object
            for row in valid_rows_filtered:
                net_flx = row['Net Flux (Jy)']
                net_flx_err = row['Flux Uncertainty']
                distance_to = 4.628516371E+23 * row['Galactic_Distance'] # GD is in Mpc, so converted it to meters
                #print(net_flx)
                luminosity, luminosity_unc = to_lum(distance_to, net_flx, net_flx_err)
                row['IR Luminosity'].append(luminosity)
                row['IR Luminosity Uncertainty'].append(luminosity_unc)

            #print('valid sources', valid_rows_filtered)


            #print( len(detectedpos_all))
            if len(detectedpos_all) > 0:
                apertures_detected = CircularAperture(detectedpos_all, r=2)
            if len(Yes) > 0:
                apertures_Yes = CircularAperture(Yes, r=YesRadius)
          
         # xc = 244.422687    #19.014239
        # yc=  191.596758# 310.340772
      
           # Plotting for current image
            '''
            fig, ax = plt.subplots(subplot_kw={'projection': wcs})
            norm = ImageNormalize(cutout.data, stretch=SqrtStretch())
            for row in valid_rows_filtered: # use 'valid_rows_filtered' to change it back to only the valid sources. use 'rows' to plot all sources
                target_aperture = CircularAperture((row['X'], row['Y']), row['Radius'])
                annulus_aperture = CircularAnnulus((row['X'], row['Y']), row['Annulus_Inner_Radius'], row['Annulus_Outer_Radius'])
                target_aperture.plot(color='red', lw=1.5, alpha=.5, ax=ax)
                annulus_aperture.plot(color='blue', lw=1.5, alpha=.5, ax=ax)
              
                apertures.plot(color='#98ff98', lw=.5, alpha=0.5)
                apertures_detected.plot(color='#FF4500', lw=.5, alpha=0.5)  #  #FF69B4 for hot pink
                apertures_Yes.plot(color='green', lw=.5, alpha=0.5)


                # curious ones
            # curious = CircularAperture((xc,yc),5)
            # curious.plot(color='red', lw=.5, alpha=0.5)
            ax.imshow(cutout.data, cmap='gray', norm=norm, interpolation='nearest')
            ax.set_xlabel('Right Ascension')
            ax.set_ylabel('Declination')
            plt.title(f'Band {band_labels[band_id]}')
            plt.show()
            yesses =[row for row in valid_rows_filtered if row['Exists?'] == 'Yes!!']
            #'''
   
display_data = pd.DataFrame(rows)
#by_name = display_data.groupby('Galaxy Name')
#byname = by_name.apply(lambda x: x).reset_index(drop=True)
#print("\n FINAL Sources Grouped by Galaxy:")
#display(byname)
#byname_byband = by_name.groupby('band_id')
#display(byname_byband)

#pd.set_option("display.max_rows", None)
#pd.set_option("display.max_columns", None)

print('Number of valid sources: ',len(display_data.loc[display_data['Flag']== 'Valid']))
print('Number of sources that dont 100 percent overlap x4 (because it is iterating over all 4 bands and appending each one once): ',len(display_data))
print('Number of sources coincidental with WISE bright points within 5 arcsec: ',len(display_data.loc[display_data['Exists?']== 'Yes!!']))
'''
print("\n all of the valid sources")
display(display_data.loc[display_data['Flag']== 'Valid'])
print("\n all of the Yes!! sources")
display(display_data.loc[display_data['Exists?']== 'Yes!!'])

display(display_data)
print(len(display_data.loc[display_data['Flag']== 'Valid']))
  
#'''



In [ ]:
# Function to convert list columns to strings
def convert_list_columns_to_strings(df):
    for col in df.columns:
        if df[col].apply(type).eq(list).any():
            df[col] = df[col].apply(str)
    return df




#FOR SOURCES COINCIDENT WITH XRAY
# lots of them output as lists and sets
# band id's are sets, a few others are lists which are unhashable

# convert the "set" tyoe of band_id to string :
# Convert the set values in 'band_id' to strings
exists =  display_data.loc[display_data['Exists?']== 'Yes!!']
exists['band_id'] = exists['band_id'].apply(lambda x: list(x)[0] if isinstance(x, set) else x)
# Convert list columns to strings
exists = convert_list_columns_to_strings(exists)

data_out = Table.from_pandas(exists)
output_path = '../Data/Hugefiles/forced_phot_YES1.csv'

# Write the table in csv format
data_out.write(output_path, format='csv')


## Trying to incorporate Abs Mag

In [ ]:
####Defining the constants
# defining a function to calculate the distances between two sources.
def dist(p1, p2):
   return np.sqrt( (p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )


#defining a function that creates a circular mask around each source so that if something overlaps with it, that overlapping part is not included in the aperture photometry
def create_circular_mask(h,w,center,radius): # I did not end up needing this code.
   Y, X = np.ogrid[:h, :w] # creating an open (more memory efficient) coordinate grid of the image
   dist_from_center = np.sqrt((X-center[0])**2+ (Y-center[1])**2)
   mask = dist_from_center <= radius # so that everything inside the radius receives a mask
   return mask

#defining a function for the overlap area so that I can scale the overlap counts with it.
def overlap_area(radius, distance):
    Area = ((2 * radius**2) * np.arccos((distance / (2 * radius)))) - ( .5 * distance * np.sqrt( (4 * radius**2) - distance**2))
    return Area

# defining a function to convert fluxes to luminosity.  L=4πr2f.
def to_lum(r, flux, net_flx_err):
    luminosity = 4 * math.pi * r * 2 * flux
    luminosity_unc = 4 * math.pi * r**2 * net_flx_err
    return luminosity, luminosity_unc



# define a mapping of the bands into labels to make it easier
band_labels = {'w1': 'W1', 'w2': 'W2', 'w3': 'W3', 'w4': 'W4'}
flux_zmfd = {'w1': 309.54 ,'w2': 171.787,'w3': 31.674,'w4': 8.363} # check if these worked by looking at the band 4 code above
instr_zpmag = {'w1': 20.73,'w2': 19.56,'w3': 17.6 ,'w4':12.98 }
wavelengths = {'w1': 3.4 ,'w2': 4.6,'w3': 12,'w4': 22}


#define function to get flux density per unit frequency (energy units)
def flux_dens(net_flx, net_flx_err, wavelength):
   flux_density = (net_flx * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   flux_density_unc = (net_flx_err * 1e-23) * (3e10 / (wavelength*1e-4)**2)
   return flux_density, flux_density_unc


#import huge csv and grab the name and ra and dec needed for each galaxy.
targetgals = pd.read_csv('../Data/inWISE.csv') # this should not be the one for all 120 and should rather be for the 74 of them.
#print(targetgals[0:20])
huge = pd.read_csv('../Data/Hugefiles/Source_Flux_All_Modified_5.csv')
columns = ['RA','Dec','Gname_Modified','Gname_Homogenized', 'ObsID', 'EXPOSURE', 'NET_LUM_APER_0.3-8.0', 'Hard_Flux_Color' , 'Soft_Flux_Color', 'Galactic_Distance']
g_huge = huge[columns]
#display(g_huge.head())


#group the x-ray sources for this galaxy. locate through merging
df1 = targetgals
df2 = g_huge


merged_data = pd.merge(df1, df2, left_on='source_id', right_on = 'Gname_Homogenized', how='inner')
columns1 = ['RA','Dec','Gname_Homogenized', 'ObsID', 'EXPOSURE', 'NET_LUM_APER_0.3-8.0', 'Hard_Flux_Color' , 'Soft_Flux_Color', 'Galactic_Distance']
Xray_sources = merged_data[columns1]

#group by galaxy name and the longest exposure time.
longest_exposure_obs = Xray_sources.loc[Xray_sources.groupby('Gname_Homogenized')['EXPOSURE'].idxmax()]

# aggregate all the sources associated with the obsid with the longest exposure time
aggregated_sources = Xray_sources[Xray_sources['ObsID'].isin(longest_exposure_obs['ObsID'])]

#create a list of all the names needed
galaxy_names = targetgals['source_id'].unique()

galaxy_sources = {}
grouped_sources = aggregated_sources.groupby('Gname_Homogenized')
'''print("\nGrouped sources:")
for group_name, group in grouped_sources:
   print(f"\nGroup: {group_name}")
   print(group)'''
#get all of the ra and dec sources for the galaxy in question
for group_name, group in grouped_sources:
   galaxy_sources[group_name] = {'ra' : group['RA'].values, 'dec' : group['Dec'].values, 'ObsID': group['ObsID'].values, 
                                 'NET_LUM_APER_0.3-8.0': group['NET_LUM_APER_0.3-8.0'].values,  'Hard_Flux_Color': group['Hard_Flux_Color'].values , 'Soft_Flux_Color': group['Soft_Flux_Color'].values, 'Galactic_Distance': group['Galactic_Distance'].values }
  
rows = []
#create subset for testing:

targetgals_subset = targetgals.iloc[0:5]

# Lookup and define a service for ALLWISE Atlas images
allwise_service = vo.dal.SIAService("https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?")

#loop through the galaxies
#print("\nAligned target galaxies and grouped sources:")
for galaxy in targetgals.itertuples():
    galaxy_name = galaxy.source_id
    #print(galaxy_name)
    #group = aligned_aggregatedsources.get_group(galaxy_name)
  
    # Print galaxy information
    #print(f"\nGalaxy: {galaxy_name}")
    #print(group)

    #define coordinates
    ra1 = galaxy.ra_x
    #print (ra1)
    dec1 = galaxy.dec_x
    pos = SkyCoord(ra=ra1, dec=dec1, unit= 'deg')
    #search the service for images covering within 1 arcsecond of the star. make this bigger if needed
    im_table = allwise_service.search(pos=pos, size= 1*u.arcsec)
    #im_table
    im_table.to_table().colnames
    #print(im_table)
    # get the Ra and dec values necessary for the kdtree and rest of the code
    if galaxy_name in galaxy_sources:
       ra = galaxy_sources[galaxy_name]['ra']
       dec = galaxy_sources[galaxy_name]['dec']
       obsid = galaxy_sources[galaxy_name]['ObsID']
       NET_LUM_APER_all = galaxy_sources[galaxy_name]['NET_LUM_APER_0.3-8.0']
       Hard_Flux_Color = galaxy_sources[galaxy_name]['Hard_Flux_Color']
       Soft_Flux_Color = galaxy_sources[galaxy_name]['Soft_Flux_Color']
       Galactic_Distance = galaxy_sources[galaxy_name]['Galactic_Distance']
       print(f"Galaxy: {galaxy_name}")
       #print("RA values:", ra)
       #print("Number of RA values:", len(ra))
    else:
       print(f"No sources found for galaxy: {galaxy_name}")
       continue # skip to the next galaxy if no sources were found
          
   ##running the for loop over every image and doing aperture photometry on each one
   #currently outputs as w4,w1,w2,w3 when querying the images. so index is 0.1.2.3 i want the index to be 0.3.2.1

    for i in [0, 3, 2, 1]:  # index is different for every single image, going to keep this anyways
        band_id = im_table[i]["sia_bp_id"].lower()  # Get band ID in lowercase
        if band_id in band_labels:
            #print(f'Band {band_labels[band_id]}: ')
            data_url = im_table[i].getdataurl()
            #Download the image and open it in Astropy
            fname = download_file(data_url, cache=True)
            image1= fits.open(fname)
            image_data= image1[0].data
            #print(data)
            #print(data_url)
            wcs = WCS(image1[0].header)
            #cuting out the image of the galaxy apart from the rest of the background.
            cutout = Cutout2D(image_data, pos, (437,437), wcs=wcs)
            wcs = cutout.wcs
            cutout_data = cutout.data
            #print(cutout_data)
            positions = wcs.world_to_pixel_values(ra, dec)
            positions = np.array(list(zip(positions[0], positions[1])))

            #define the distance threshold for the KDTree grouping (in pixels). 5 usually works best. any larger and the average of their position goes wonky
            distance_threshold = 5

            #build the KDTree for efficient grouping
            tree = KDTree(positions)

            #query the KDTree to find points within the defined radius of dist threshold and group them together
            groups = tree.query_ball_tree(tree, r=distance_threshold)
            # print(groups)
            # consolidating the groups. 'unique_groups' and 'seen': These are used to ensure that each group is processed only once.
            unique_groups = []
            seen = set()
            for group in groups:
                group = tuple(sorted(group))
                if group not in seen:
                    seen.add(group)
                    unique_groups.append(group)
                 # print(unique_groups)
            # for each unique group, the average postion of the detections is calulated so that only one source detection is used for aperture photometry instead of a bunch of the same sources being used.
             #represents the consolidated postion of potentially multiple detections of one source.
            grouped_positions = [positions[list(group)].mean(axis=0) for group in unique_groups]
            #print(grouped_positions)

            #print("Grouped positions for galaxy", galaxy_name, ":", grouped_positions)
            #define the Region(s) Of Interest (center x, center y, radius)
            ROI = [ ((x, y) , 9, 16, 23) for x,y in grouped_positions ] # (x, y, radius around target, inner r, outer r)   36.3636, 50.90909) may need to mke the radius bigger when goruping?
          
                # initialize valid rows plotting for the current image iteration
            valid_rows = []
           
           #now inputting the aperture photometry part
           # check for overlap and perform aperture photometry
            for i, ((x, y), r, annulus_inner, annulus_outer) in  enumerate(ROI):
                overlap_dict = []
                overlap = False # initialize overlap flag (A boolean flag overlap is set to False for each source to track if it overlaps with any other source. becomes true if they do overlap 
                acc_overlap = False # initialize the acceptable overlap flag.  false if there is no overlap, true if there is overlap and it is acceptable 
               
                for j, ((x2, y2), r2, annulus_inner2, annulus_outer2) in  enumerate(ROI): # apparently you can run a for loop over 2 objects by putting the second one inside the first. it iterates over every source again to then see if it overlaps at all with another source.
                    if i != j: # ensures that a source is not compared to itself! wow
                        #print(f'{x}, {y} / {x2}, {y2}')
                        #print(f"Checking positions: ({x}, {y}) and ({x2, {y2}})")
                        distance = dist((x, y) , (x2, y2))
                        #print(f"Distance: {distance}, Radii Sum: {r + r2}")
                        #print('dsitance', distance)
                        #print('Distance', distance)
                        #print('r1', r)
                        if distance < r + r2:  # if the distance is less than the size of the two radii added together, then they are overlapping.
                            #print(distance)
                            #print('yesif')
                            overlap_percent = (r + r2 - distance)/(r+r2)  # the amount they are overlapping divided by the total size of radii distance
                            #print('overlap perc', overlap_percent)
                            if overlap_percent > .5:
                                overlap = True # this way, if they overlap by more than 50% then they will not be usable because less than 50% of the flux extractable area can be seen.
                                #print('overlap is unacceptable: ', overlap)
                                overlap_aperture = np.nan
                                overlap_photo_table = np.nan
                                overlap_counts = np.nan
                                overlap_error = np.nan
                              
                            elif overlap_percent <= .5:
                                acc_overlap = True
                                #print('acceptable or no overlap: ', acc_overlap)
                                #Handle overlaps that are acceptable (less than the threshold, but still overlapping by a small percent)
                                overlap_aperture = CircularAperture((x2, y2), r2)
                                overlapping_area = overlap_area(r,distance) # using the function to define the overlapping area for the location of overlap
                                overlap_photo_table = aperture_photometry(cutout_data, overlap_aperture)
                                total_area = math.pi * r**2
                                overlap_counts = overlap_photo_table['aperture_sum'][0] * ( overlapping_area/total_area ) # scaling the counts by the overlapping area
                                overlap_error = np.sqrt(overlap_counts)
                                overlap_dict.append({'Position': ({x}, {y}),  'overlapping counts': overlap_counts, 'overlap_error': overlap_error})
                                #print('were here')
                                #print('Overlap counts', overlap_counts)
                                #print('overlapdict', overlap_dict)

                        else:
                            #print(' Prob skips distance if statement')
                            #print('not overlapping at all')
                            overlap_percent = np.nan
                            overlap_aperture = np.nan
                            overlap_photo_table = np.nan
                            overlap_counts = np.nan
                            overlap_error = np.nan
                       
                #print(acc_overlap)  
                if acc_overlap:
                    overlap_counts = 0
                    overlap_error = 0
                    # for a source, if it overlaps with more than one other source, add all of the counts for those overlapping regions
                    for row in overlap_dict:
                        if not np.isnan(row['overlapping counts']) and not np.isnan(row['overlap_error']):
                            overlap_counts += row['overlapping counts'] # now add in quadriture for the propagation of error for sources like this
                            overlap_error += row['overlap_error']**2
                    overlap_error= np.sqrt(overlap_error)
                    #print('Overlapping COUNTS HERE', overlap_counts)
                    # For the Target objects in the little aperture circle define their target apertures
                    target_aperture = CircularAperture((x,y),r,)
                   
                    #perform aperture photometry on target
                    target_photo_table = aperture_photometry(cutout_data, target_aperture)
                    target_counts = target_photo_table['aperture_sum'][0]

                    target_counts -= overlap_counts
                    # continuing on with the photometry under the "if acc_overlap"

                    #calculate area of target aperutue
                    target_area = target_aperture.area
                    # For the Background Annuli of outside of the Target
                    #define the background annulus for the target
                    annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)

                    #perform aperture photometry on annuli
                    annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                    annulus_counts = annulus_photo_table['aperture_sum'][0]
                    ann_err = np.sqrt(annulus_counts)
                    #background subtraction for just Mcal
                    # Calculate the background counts per pixel

                    background_mean = annulus_photo_table['aperture_sum'][0] / annulus_aperture.area
                    bg_mean_err = ann_err / annulus_aperture.area
                    # Subtract the background from the target counts
                    counts_for_mcal = target_counts - (background_mean * target_aperture.area)
                    
                    #for abs mag calculation
                    if counts_for_mcal > 0: #
                        target_counts_err1 = np.sqrt(target_counts)
                        target_counts_err2 = np.sqrt(target_counts_err1**2 + overlap_error**2)
                        counts_for_mcal_err = np.sqrt(target_counts_err2**2 + (bg_mean_err * target_aperture.area)**2)
                        #propagated error of overlap error
                        mcal_overlap_counts_err = np.sqrt(counts_for_mcal_err**2 + overlap_error**2)
                        #print(target_counts)
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            mcalib = M0instr - 2.5*(np.log10(counts_for_mcal))     #converting counts to calibrated magnitude
                            
                            #propagation of uncertainty 
                            Mcal_error = (2.5*mcal_overlap_counts_err) / (counts_for_mcal * np.log(10))

                            # convert calibrated magnitude to absolute magnitude, dont worry abt error rn
                            abs_mag = mcalib - 5 * np.log10((Galactic_Distance[0] * 1e6) / 10)
                            abs_mag_unc = Mcal_error - 5 * np.log10((Galactic_Distance[0] * 1e6) / 10) # and propagation of error 

                    else:# to handle the cases where the counts are not more than 0 and if the conversion factors are missing.

                        target_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        abs_mag = np.nan
                        abs_mag_unc = np.nan
                        mcalib = np.nan
                        Mcal_error = np.nan

                  
                

                    #for net flux
                    # so that i dont take the log or sqrt of a negative number or zero and get an error
                    if target_counts > 0: #
                        target_error= np.sqrt(target_counts)
                        #propagated error of overlap error
                        target_overlap_counts_err = np.sqrt(target_error**2 + overlap_error**2)
                        #print(target_counts)
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to calibrated magnitude
                            target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux (looking back now this might just be flux density)

                            #propagation of uncertainty of flux conversion
                            Mcal_error = (2.5*target_overlap_counts_err) / (target_counts * np.log(10))
                            target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)

                    else:# to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        target_error = np.nan
                        target_overlap_counts_err = np.nan
                        target_flux = np.nan
                        target_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        target_counts = np.nan
                  

                    
              
                    if annulus_counts > 0:
                        overlapannulus_error = np.sqrt(annulus_counts) # the error of the annulus for sources that overlap
                        # to avoid taking the log of zero or negative value
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                            annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                            #propagation of uncertainty of flux conversion
                            Mcal_error_ann = (2.5 * overlapannulus_error) / (annulus_counts * np.log(10))
                            annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)



                    else: 
                        annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        overlapannulus_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        annulus_counts = np.nan
                          
                    #calculate area of annulus
                    annulus_area = annulus_aperture.area

                    # do the calculations for including a Background aperture
              
                    #Calculating the net flux:
                    #calculate the mean background per pixel
                    bg_perpixel = annulus_flux/annulus_area
                    bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!

                    #calculate the total background in the target aperture
                    tot_bg = bg_perpixel * target_area
                    tot_bg_err = bg_perpixel_err * target_area ##propagation of error!

                    #Subtract background from the target flux
                    net_flx = target_flux - tot_bg
                    net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!
              
                    #flag the sources that overlap
                    rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i],
                                 'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [], 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner,
                           'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,'Flux Uncertainty': net_flx_err, 'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux',
                            'Apparent Magnitude' : mcalib, 'Apparent Magnitude Uncertainty': Mcal_error, 'Absolute Magnitude': abs_mag,'Absolute Magnitude Error' : abs_mag_unc,
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux, 'Acceptable overlapping?': 'Yes' if acc_overlap == True else 'Not acceptable or no overlap at all', 'Unacceptable overlapping?': 'Yes' if overlap == True else 'Acceptable or no overlap at all',
                               'Annulus Counts': annulus_counts, 'Overlap Counts': overlap_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                    # Append valid results to valid_rows
                    valid_rows.append({
                       'band_id': {band_labels[band_id]},  'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i], 'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [],
                       'Apparent Magnitude' : mcalib, 'Apparent Magnitude Uncertainty': Mcal_error,  'Absolute Magnitude': abs_mag,'Absolute Magnitude Error' : abs_mag_unc,
                       'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                       'Annulus_Inner_Radius': annulus_inner,
                       'Annulus_Outer_Radius': annulus_outer,
                       'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux', 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                       'Offset in Arcseconds': [], 'Exists?': 'No', 'Point Source Position' : [],  'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': overlap_error, 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })
                  
                else: #perform all the normal aperture photometry stuff for those that do not overlap in any way.
            
                   # For the Target objects in the little aperture circle define their target apertures
                    target_aperture = CircularAperture((x,y),r,)
              
                    #perform aperture photometry on target
                    target_photo_table = aperture_photometry(cutout_data, target_aperture)
                    target_counts = target_photo_table['aperture_sum'][0]

                    # continuing on with the photometry under the "if acc_overlap"

                   #calculate area of target aperutue
                    target_area = target_aperture.area
                    # For the Background Annuli of outside of the Target
                    #define the background annulus for the target
                    annulus_aperture = CircularAnnulus((x, y), annulus_inner, annulus_outer)

                    #perform aperture photometry on annuli
                    annulus_photo_table = aperture_photometry(cutout_data, annulus_aperture)
                    annulus_counts = annulus_photo_table['aperture_sum'][0]
                    ann_err = np.sqrt(annulus_counts)
                    #background subtraction for just Mcal
                    # Calculate the background counts per pixel

                    background_mean = annulus_photo_table['aperture_sum'][0] / annulus_aperture.area
                    bg_mean_err = ann_err / annulus_aperture.area
                    # Subtract the background from the target counts
                    counts_for_mcal = target_counts - (background_mean * target_aperture.area)
                    
                    #for abs mag calculation
                    if counts_for_mcal > 0: #
                        target_counts_err1 = np.sqrt(target_counts)
                        target_counts_err2 = np.sqrt(target_counts_err1**2 + overlap_error**2)
                        counts_for_mcal_err = np.sqrt(target_counts_err2**2 + (bg_mean_err * target_aperture.area)**2)
                        #propagated error of overlap error
                        mcal_overlap_counts_err = np.sqrt(counts_for_mcal_err**2 + overlap_error**2)
                        #print(target_counts)
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            mcalib = M0instr - 2.5*(np.log10(counts_for_mcal))     #converting counts to calibrated magnitude
                            
                            #propagation of uncertainty 
                            Mcal_error = (2.5*mcal_overlap_counts_err) / (counts_for_mcal * np.log(10))

                            # convert calibrated magnitude to absolute magnitude, dont worry abt error rn
                            abs_mag = mcalib - 5 * np.log10((Galactic_Distance[0] * 1e6) / 10)
                            abs_mag_unc = Mcal_error - 5 * np.log10((Galactic_Distance[0] * 1e6) / 10) # and propagation of error 

                    else:# to handle the cases where the counts are not more than 0 and if the conversion factors are missing.

                        target_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        abs_mag = np.nan
                        abs_mag_unc = np.nan
                        mcalib = np.nan
                        Mcal_error = np.nan


                    if target_counts > 0:   # avoid taking the log of zero or negative value
                        target_error = np.sqrt(target_counts)
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(target_counts))     #converting counts to magnitude
                            target_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                            #propagation of uncertainty of flux conversion
                            Mcal_error = (2.5*target_error) / (target_counts * np.log(10))
                            target_flux_error = target_flux * np.log(10) * (Mcal_error/2.5)
                            
                            # convert calibrated magnitude to absolute magnitude, dont worry abt error rn
                            Mcal_trgt_test = M0instr - 2.5*(np.log10(target_counts)) 
                            abs_mag = Mcal_trgt_test - 5 * np.log10((Galactic_Distance[0] * 1e6) / 10)
                            abs_mag_unc = Mcal_error + (-5 * np.log((Galactic_Distance[0]*1e6)/10)) # and propagation of error 

                    else:
                        target_error = np.nan
                        target_flux = np.nan
                        target_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        target_counts = np.nan
                        abs_mag = np.nan
                        abs_mag_unc = np.nan
                        Mcal_trgt_test = np.nan

                    
                  
                    if annulus_counts > 0:
                        annulus_error = np.sqrt(annulus_counts)
                        # avoid taking the log of zero or negative value
                        if band_id in flux_zmfd and instr_zpmag:
                            #print(f'Band {flux_zmfd[band_id]}: ')
                            flx_conv_fact = flux_zmfd[band_id]
                            M0instr = instr_zpmag[band_id]
                            Mcal_trgt = M0instr - 2.5*(np.log10(annulus_counts))     #converting counts to magnitude
                            annulus_flux = flx_conv_fact * 10**(Mcal_trgt/-2.5)      #convert Magnitude to Flux

                            #propagation of uncertainty of flux conversion
                            Mcal_error_ann = (2.5 * annulus_error) / (annulus_counts * np.log(10))
                            annulus_flux_error = annulus_flux * np.log(10) * (Mcal_error_ann/2.5)

                    else:
                        annulus_flux = np.nan # to handle the cases where the counts are not more than 0 and if the conversion factors are missing.
                        annulus_error = np.nan
                        annulus_flux_error = np.nan
                        flx_conv_fact = np.nan
                        M0instr = np.nan
                        annulus_counts = np.nan
                        
                          
                    #calculate area of annulus
                    annulus_area = annulus_aperture.area

                    # do the calculations for including a Background aperture
              
                    #Calculating the net flux:
                    #calculate the mean background per pixel
                    bg_perpixel = annulus_flux/annulus_area
                    bg_perpixel_err = annulus_flux_error/annulus_area #propagation of error!

                    #calculate the total background in the target aperture
                    tot_bg = bg_perpixel * target_area
                    tot_bg_err = bg_perpixel_err * target_area ##propagation of error!

                    #Subtract background from the target flux
                    net_flx = target_flux - tot_bg
                    net_flx_err = np.sqrt(target_flux_error**2 + tot_bg_err**2) ##propagation of error!

                    if overlap:
                        overlap_counts = np.nan
                        overlap_error = np.nan
                        target_counts = np.nan

                    #   Append the result as a dictionary to the list named 'rows'
                    rows.append({ 'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i], 
                                 'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [],
                                 'Apparent Magnitude' : mcalib, 'Apparent Magnitude Uncertainty': Mcal_error, 'Absolute Magnitude': abs_mag,'Absolute Magnitude Error' : abs_mag_unc,
                                 'Region': i+1, 'X': x, 'Y': y, 'Radius': r, 'Annulus_Inner_Radius': annulus_inner, 'Annulus_Outer_Radius': annulus_outer, 'Net Flux (Jy)': net_flx,
                                   'Flux Uncertainty': net_flx_err,  'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux',
                           'aperture_sum': target_photo_table['aperture_sum'][0] ,'tot_bg': tot_bg, 'Target Counts': target_counts, 'Target Flux': target_flux, 'Acceptable overlapping?': 'Yes' if acc_overlap == True else 'Not acceptable or no overlap at all','Unacceptable overlapping?': 'Yes' if overlap == True else 'Acceptable or no overlap at all',
                               'Annulus Counts': annulus_counts, 'Annulus Flux': annulus_flux,'Image Data Shape': cutout_data.shape, 'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr,
                               'Offset in Arcseconds': [],   'Exists?': 'No', 'Point Source Position' : [], 'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id],'Flux Density': [], 'Flux Density Uncertainty' : [] })  #will prolly have to change the name of the Flux here!!!
                        #"Low Flux" means that they either have zero flux or negative flux and so they are excluded from the plotting
              
                    # Append valid results to valid_rows
                    valid_rows.append({
                       'band_id': {band_labels[band_id]}, 'Galaxy Name' : galaxy_name, 'ObsID' : obsid[0], 'NET_LUM_APER_0.3-8.0' : NET_LUM_APER_all[i], 'Hard_Flux_Color' : Hard_Flux_Color[i], 'Soft_Flux_Color' : Soft_Flux_Color[i],'Galactic_Distance': Galactic_Distance[0], 'IR Luminosity': [], 'IR Luminosity Uncertainty': [],
                       'Apparent Magnitude' : mcalib, 'Apparent Magnitude Uncertainty': Mcal_error,   'Absolute Magnitude': abs_mag,'Absolute Magnitude Error' : abs_mag_unc,
                       'Region': i+1, 'X': x, 'Y': y, 'Radius': r,
                       'Annulus_Inner_Radius': annulus_inner,
                       'Annulus_Outer_Radius': annulus_outer,
                       'Net Flux (Jy)': net_flx, 'Flux Uncertainty': net_flx_err,
                       'Flag':'Valid' if not np.isnan(np.array(target_counts)) and target_counts > 0 and net_flx > 0 else 'Nan values or Low Flux',
                       'Flux Conv':flx_conv_fact, 'MzpInstr':M0instr, 'Exists?': 'No', 'Point Source Position' : [], 'Offset in Arcseconds': [],
                       'Target Error': target_flux_error, 'Annulus Error': annulus_flux_error, 'Overlap Error (in counts)': [], 'Wavelength': wavelengths[band_id], 'Flux Density': [], 'Flux Density Uncertainty' : [] })

            #Source detection code
            mean, median, std = sigma_clipped_stats(cutout_data, sigma=3.0) 
            #print((mean, median, std))

            # subtract the background and find FWHM of sources at a certain threshold
            #started at fwhm= 3 and threshold = 5
            daofind = DAOStarFinder(fwhm=8, threshold=1*std) # find the stars in the image that have FWHMs of around 3 pixels and have peaks approximately 5-sigma above the background.
            sources = daofind(cutout_data - median) 
            #print(type(sources))
            # will likely run into iissues in the code below
            for col in sources.colnames: 
                if col not in ('id', 'npix'):
                    sources[col].info.format = '%.2f'  # for consistent table output
          
            # plot the image with marked locations of all the sources it detected.
            detected_positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
            apertures = CircularAperture(detected_positions, r=2)

            # Plotting for current image
            # Filter valid rows
            valid_rows_filtered = [row for row in valid_rows if row['Flag'] == 'Valid']
          
            # Was there a point source there? to within a certain numb of arcseconds
            pixelsinarc = 0.0003819444391411 * 3600 ## 0.0003819444391411 is the number found in the header of the image for the scale of pixels in degrees for the fits image.
            detectedpos_all = []
            #rows = [row for row in rows if row['Flag'] == 'Valid']
            for row in valid_rows_filtered:
                apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
                for detected_position in detected_positions:
                    detected_x, detected_y = detected_position
                    distance = dist((row['X'], row['Y']), (detected_x, detected_y) ) #distance from the center of the xray source to the center of the detected source
                    if distance <= row['Radius']:
                        row['Exists?'] = 'Point Source Detected'
                        row['Point Source Position'].append((detected_x, detected_y))
                        dist_in_arc = distance * pixelsinarc
                        row['Offset in Arcseconds'].append((dist_in_arc))
                        detectedpos_all.append(detected_position)
            # used for creating apertures for sources within the radius of the X-ray source apertures.
            Yes = []
            YesRadius= 5
            for row in valid_rows_filtered:
                        apertures_forced = CircularAperture((row['X'], row['Y']), r=row['Radius'])
                        for detected_position in detected_positions:
                            detected_x, detected_y = detected_position
                            distance = dist((row['X'], row['Y']), (detected_x, detected_y) )
                            if distance <= YesRadius:
                                row['Exists?'] = 'Yes!!'
                                Yes.append((row['X'], row['Y']))
                                #else: # this does not seem to do what i want it to do
                               #row['Exists?'] = 'Detected but does not exist'
            '''       
            Did not realize that the flux conversion i was doing was into flux density, so this part is no longer needed.            
            #doing flux density
            for row in valid_rows_filtered:
                net_flx = row['Net Flux (Jy)']
                net_flx_err = row['Flux Uncertainty']
                wavelength = row['Wavelength']
                #print(net_flx)
                flux_density, flux_density_unc = flux_dens(net_flx, net_flx_err, wavelength)
                row['Flux Density'].append(flux_density)
                row['Flux Density Uncertainty'].append(flux_density_unc)
                  '''
            # Update rows with valid_rows_filtered information
            for valid_row in valid_rows_filtered:
                for row in rows:
                    if row['band_id'] == valid_row['band_id'] and row['Region'] == valid_row['Region']:
                        row.update(valid_row)

            # Converting to luminosity  L=4πr2f. where r = distance to the object
            for row in valid_rows_filtered:
                net_flx = row['Net Flux (Jy)']
                net_flx_err = row['Flux Uncertainty']
                distance_to = 4.628516371E+23 * row['Galactic_Distance'] # GD is in Mpc, so converted it to meters
                #print(net_flx)
                luminosity, luminosity_unc = to_lum(distance_to, net_flx, net_flx_err)
                row['IR Luminosity'].append(luminosity)
                row['IR Luminosity Uncertainty'].append(luminosity_unc)

            #print('valid sources', valid_rows_filtered)


            #print( len(detectedpos_all))
            if len(detectedpos_all) > 0:
                apertures_detected = CircularAperture(detectedpos_all, r=2)
            if len(Yes) > 0:
                apertures_Yes = CircularAperture(Yes, r=YesRadius)
          
         # xc = 244.422687    #19.014239
        # yc=  191.596758# 310.340772
      
           # Plotting for current image
            '''
            fig, ax = plt.subplots(subplot_kw={'projection': wcs})
            norm = ImageNormalize(cutout.data, stretch=SqrtStretch())
            for row in valid_rows_filtered: # use 'valid_rows_filtered' to change it back to only the valid sources. use 'rows' to plot all sources
                target_aperture = CircularAperture((row['X'], row['Y']), row['Radius'])
                annulus_aperture = CircularAnnulus((row['X'], row['Y']), row['Annulus_Inner_Radius'], row['Annulus_Outer_Radius'])
                target_aperture.plot(color='red', lw=1.5, alpha=.5, ax=ax)
                annulus_aperture.plot(color='blue', lw=1.5, alpha=.5, ax=ax)
              
                apertures.plot(color='#98ff98', lw=.5, alpha=0.5)
                apertures_detected.plot(color='#FF4500', lw=.5, alpha=0.5)  #  #FF69B4 for hot pink
                apertures_Yes.plot(color='green', lw=.5, alpha=0.5)


                # curious ones
            # curious = CircularAperture((xc,yc),5)
            # curious.plot(color='red', lw=.5, alpha=0.5)
            ax.imshow(cutout.data, cmap='gray', norm=norm, interpolation='nearest')
            ax.set_xlabel('Right Ascension')
            ax.set_ylabel('Declination')
            plt.title(f'Band {band_labels[band_id]}')
            plt.show()
            yesses =[row for row in valid_rows_filtered if row['Exists?'] == 'Yes!!']
            #'''
   
display_data = pd.DataFrame(rows)
#by_name = display_data.groupby('Galaxy Name')
#byname = by_name.apply(lambda x: x).reset_index(drop=True)
#print("\n FINAL Sources Grouped by Galaxy:")
#display(byname)
#byname_byband = by_name.groupby('band_id')
#display(byname_byband)

#pd.set_option("display.max_rows", None)
#pd.set_option("display.max_columns", None)

print('Number of valid sources: ',len(display_data.loc[display_data['Flag']== 'Valid']))
print('Number of sources that dont 100 percent overlap x4 (because it is iterating over all 4 bands and appending each one once): ',len(display_data))
print('Number of sources coincidental with WISE bright points within 5 arcsec: ',len(display_data.loc[display_data['Exists?']== 'Yes!!']))
'''
print("\n all of the valid sources")
display(display_data.loc[display_data['Flag']== 'Valid'])
print("\n all of the Yes!! sources")
display(display_data.loc[display_data['Exists?']== 'Yes!!'])

display(display_data)
print(len(display_data.loc[display_data['Flag']== 'Valid']))
  
#'''



In [7]:
exists =  display_data.loc[display_data['Exists?']== 'Yes!!']
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)
display(exists.head(30))

,band_id,Galaxy Name,ObsID,NET_LUM_APER_0.3-8.0,Hard_Flux_Color,Soft_Flux_Color,Galactic_Distance,IR Luminosity,IR Luminosity Uncertainty,Apparent Magnitude,Apparent Magnitude Uncertainty,Absolute Magnitude,Absolute Magnitude Error,Region,X,Y,Radius,Annulus_Inner_Radius,Annulus_Outer_Radius,Net Flux (Jy),Flux Uncertainty,Flag,aperture_sum,tot_bg,Target Counts,Target Flux,Acceptable overlapping?,Unacceptable overlapping?,Annulus Counts,Annulus Flux,Image Data Shape,Flux Conv,MzpInstr,Offset in Arcseconds,Exists?,Point Source Position,Target Error,Annulus Error,Overlap Error (in counts),Wavelength,Flux Density,Flux Density Uncertainty,Overlap Counts
0,{W4},NGC 4417,14902,NaN,0.542075,NaN,16.8,[1.4512175597374828e+22],[1.1387905984194468e+49],12.629051,0.004434,-30.091299,-71.667088,1,399.211027,142.185253,9,16,23,0.000074,0.014988,Valid,NaN,NaN,NaN,NaN,Not acceptable or no overlap at all,Acceptable or no overlap at all,NaN,NaN,"(437, 437)",8.363,12.98,[6.669909074600799],Yes!!,"[(403.8203804675527, 143.69672141155763)]",0.013162,0.024163,[],22.0,[],[],NaN
1,{W4},NGC 4536,19387,NaN,0.507839,NaN,13.3,[4.4429174342242915e+23],[7.329640536781079e+48],8.660568,0.004317,-29.641986,-70.499131,2,377.933601,22.318875,9,16,23,0.002872,0.015392,Valid,NaN,NaN,NaN,NaN,Not acceptable or no overlap at all,Acceptable or no overlap at all,NaN,NaN,"(437, 437)",8.363,12.98,[5.914647816374392],Yes!!,"[(381.2870034012018, 25.012967011813853)]",0.013518,0.024806,[],22.0,[],[],NaN
3,{W4},NGC 1792,19524,1.270832e+39,0.748063,NaN,13.6,[4.0303816787911e+25],[3.7709795569068076e+48],3.790591,0.008479,-28.224503,-70.606497,4,431.947871,47.526962,9,16,23,0.254757,0.007573,Valid,NaN,NaN,NaN,NaN,Not acceptable or no overlap at all,Acceptable or no overlap at all,NaN,NaN,"(437, 437)",8.363,12.98,[1.0431493707960338],Yes!!,"[(432.296387958713, 48.20082533515143)]",0.006882,0.010653,[],22.0,[],[],NaN
6,{W4},NGC 1792,19524,2.597965e+38,0.564361,NaN,13.6,[1.5244654824497198e+23],[4.341640907319487e+48],9.846163,0.007620,-28.456415,-70.607356,7,344.462348,312.508632,9,16,23,0.000964,0.008719,Valid,36555.702476,1.965070,31053.060598,1.669039,Yes,Acceptable or no overlap at all,123223.488562,6.623013,"(437, 437)",8.363,12.98,[1.2616300941124432],Yes!!,"[(345.0216890950249, 313.23597936091664)]",0.007658,0.014052,[],22.0,[],[],5502.641878
8,{W4},NGC 1792,19524,2.059180e+39,0.619129,-0.413957,13.6,[9.638553830204083e+22],[4.340032623596638e+48],10.343927,0.007624,-28.455530,-70.607353,9,338.764515,46.865411,9,16,23,0.000609,0.008716,Valid,36357.257809,1.954377,29233.515785,1.571242,Yes,Yes,122552.987822,6.586975,"(437, 437)",8.363,12.98,[1.2527023688413876],Yes!!,"[(338.70743900678855, 45.956144481847126)]",0.007655,0.014049,[],22.0,[],[],7123.742023
10,{W4},NGC 4302,19392,1.086925e+38,-1.000000,1.000000,16.8,[3.0159285928085816e+23],[1.0515511680332481e+49],9.334830,0.004801,-29.918366,-71.666720,11,206.680248,307.479805,9,16,23,0.001543,0.013839,Valid,36359.454599,1.954484,31608.385489,1.698887,Yes,Yes,122559.695078,6.587335,"(437, 437)",8.363,12.98,"[7.499251133277532, 2.95828705252135]",Yes!!,"[(201.35746805525076, 308.668981595912), (204....",0.012154,0.022307,[],22.0,[],[],4751.069110
12,{W4},NGC 1792,19524,8.216812e+39,0.470134,-0.531399,13.6,[2.927213518790071e+22],[4.34145771718174e+48],11.637821,0.007621,-28.456146,-70.607355,13,296.704083,137.334462,9,16,23,0.000185,0.008719,Valid,36444.193144,1.957885,36444.193144,1.958802,Not acceptable or no overlap at all,Acceptable or no overlap at all,122772.948957,6.598797,"(437, 437)",8.363,12.98,[3.607245340148183],Yes!!,"[(298.5297726271688, 135.45048953672242)]",0.007657,0.014056,[],22.0,[],[],NaN
16,{W4},IC342,22482,5.838805e+41,0.110325,-0.999560,3.9,[2.2880101122303514e+26],[8.725477958746462e+47],0.549126,0.002940,-27.406197,-27.950683,17,218.056312,219.338876,9,16,23,5.043264,0.021309,Valid,36453.268083,1.960373,36453.268083,1.959289,Not acceptable or no overlap at all,Acceptable or no overlap at all,122928.9

In [ ]:
# Function to convert list columns to strings
def convert_list_columns_to_strings(df):
    for col in df.columns:
        if df[col].apply(type).eq(list).any():
            df[col] = df[col].apply(str)
    return df




#FOR SOURCES COINCIDENT WITH XRAY
# lots of them output as lists and sets
# band id's are sets, a few others are lists which are unhashable

# convert the "set" tyoe of band_id to string :
# Convert the set values in 'band_id' to strings
exists =  display_data.loc[display_data['Exists?']== 'Yes!!']
exists['band_id'] = exists['band_id'].apply(lambda x: list(x)[0] if isinstance(x, set) else x)
# Convert list columns to strings
exists = convert_list_columns_to_strings(exists)

data_out = Table.from_pandas(exists)
output_path = '../Data/Hugefiles/forced_phot_YES1.csv'

# Write the table in csv format
data_out.write(output_path, format='csv')


In [ ]:
# FOR ALL SOURCES
data_out = Table.from_pandas(display_data)
output_path = '../Data/Hugefiles/forced_phot_ALL1.csv'

# Write the table in csv format
data_out.write(output_path, format='csv')


# Color-Color Diagrams

In [ ]:
# Function to convert list columns to strings
def convert_list_columns_to_strings(df):
    for col in df.columns:
        if df[col].apply(type).eq(list).any():
            df[col] = df[col].apply(str)
    return df

#exists = pd.read_csv('../Data/Hugefiles/forced_phot_YES.csv')
exists =  display_data.loc[display_data['Exists?']== 'Yes!!']
#display(exists)
# Convert the set values in 'band_id' to strings, and same with lists
exists['band_id'] = exists['band_id'].apply(lambda x: list(x)[0] if isinstance(x, set) else x)
exists = convert_list_columns_to_strings(exists)

display( exists)
# access flux for each band
fluxw1 = exists.loc[exists['band_id'] == 'W1', 'Net Flux (Jy)'].values
fluxw2 = exists.loc[exists['band_id'] == 'W2', 'Net Flux (Jy)'].values
fluxw3 = exists.loc[exists['band_id'] == 'W3', 'Net Flux (Jy)'].values
fluxw4 = exists.loc[exists['band_id'] == 'W4', 'Net Flux (Jy)'].values
print(len(fluxw4))
'''
print("Flux W1:", fluxw1)
print("Flux W2:", fluxw2)
print("Flux W3:", fluxw3)
print("Flux W4:", fluxw4)
'''
#make sure all of the bands have the same length for their arrays since some fluxes are missing for some positions in each band.
min_length = min(len(fluxw1), len(fluxw2), len(fluxw3), len(fluxw4) )
fluxw1 = fluxw1[:min_length]
fluxw2 = fluxw2[:min_length]
fluxw3 = fluxw3[:min_length]
fluxw4 = fluxw4[:min_length]
print(len(fluxw4))

# Drop rows where either 'Hard_Flux_Color' or 'Soft_Flux_Color' is NaN
exists = exists.dropna(subset=['Hard_Flux_Color', 'Soft_Flux_Color'])

H = exists['Hard_Flux_Color'].values
S = exists['Soft_Flux_Color'].values
Lum = exists['NET_LUM_APER_0.3-8.0'].values
lum_ir = exists['IR Luminosity'].values
lum_ir = np.array(lum_ir)

# use the min length from the flux values cuz it should be the same length either way.
H = H[:min_length]
S = S[:min_length]
Lum = Lum[:min_length]
lum_ir = lum_ir[:min_length]
print(len(S))
print(len(H))
print('length of Luminosity',len(Lum))
print(len(H))
'''
#color color plot

plt.scatter(H, S, marker = 'o')
plt.xlabel('Color 1 Hard')
plt.ylabel('Color 2 Soft')
plt.title('Color-Color Diagram')
plt.grid(True)
plt.show()


# Color- luminosity plot

plt.scatter(H, Lum, marker = 'o')
plt.xlabel('Hard')
plt.ylabel('Lum')
plt.yscale('log')
plt.title('Color-Luminosity Diagram')
plt.grid(True)
plt.show()
plt.tight_layout()


#'''



# Ensure all values are positive for log scale. redifining variables so i can make them a little different
Htest = H[H > 0]
Lumtest = Lum[Lum > 0]

min_length2 = min(len(Htest), len(Lumtest))

Htest = Htest[:min_length2]

Lumtest = Lumtest[:min_length2]



#I want to replicate what jacob had for his plots

# for COLOR - LUMINOSITY
# create the figure and the gridspec
fig = plt.figure(figsize=(10, 10))
gs = fig.add_gridspec(4, 4, wspace=0.06, hspace=0.06)
fig.suptitle('Luminosty vs Hard Color Diagram with Histograms')

# Scatter plot
plot = fig.add_subplot(gs[1:4, 0:3])
plot.scatter(H, Lum, color='blue', alpha=0.6, label='X-ray Luminosity') #using luminosities in the xray
plot.scatter(H,lum_ir, color='orange', alpha = .6, label='Infrared Luminosity') #using luminosities in WISE infrared
plot.set(xlabel='Hard Color', ylabel='Luminosity',  yscale='log')
plot.xaxis.label.set_size(20)
plot.yaxis.label.set_size(20)
plot.legend()




# Histogram for H (top)
histx = fig.add_subplot(gs[0, 0:3], sharex=plot)
histx.hist(H, bins=50, color='teal', alpha=0.6)
#histx.set_xscale('log')
histx.xaxis.set_tick_params(labelbottom=False)

# Histogram for Lum (right)

y_bins_logspace = np.logspace(np.log10(Lumtest.min()), np.log10(Lumtest.max()), 50) # need to make the histogram of the y axis also in log.
histy = fig.add_subplot(gs[1:4, 3], sharey=plot)

histy.hist(Lum, bins=np.logspace(np.log10(plt.gca().get_ylim())[0], np.log10(plt.gca().get_ylim()[1]), 100), color='teal', alpha=0.6, orientation='horizontal')
histy.set_xscale('log')
histy.yaxis.set_tick_params(labelleft=False)

# Show the plot
plt.tight_layout()
plot.grid(True)
plt.show()



# for COLOR - COLOR

# create the figure and the gridspec
fig = plt.figure(figsize=(10, 10))
gs = fig.add_gridspec(4, 4, wspace=0.06, hspace=0.06)
fig.suptitle('Hard Color vs Soft Color Diagram with Histograms')

# Scatter plot
plot = fig.add_subplot(gs[1:4, 0:3])
plot.scatter(H, S, color='blue', alpha=0.6)
plot.set(xlabel='Hard Color', ylabel='Soft Color')
plot.xaxis.label.set_size(20)
plot.yaxis.label.set_size(20)


# Histogram for H (top)
histx = fig.add_subplot(gs[0, 0:3], sharex=plot)
histx.hist(H, bins=50, color='teal', alpha=0.6)
#histx.set_xscale('log')
histx.xaxis.set_tick_params(labelbottom=False)

# Histogram for Lum (right)
histy = fig.add_subplot(gs[1:4, 3], sharey=plot)

histy.hist(S, bins=50, color='teal', alpha=0.6, orientation='horizontal')
histy.yaxis.set_tick_params(labelleft=False)

# Show the plot
plt.tight_layout()
plot.grid(True)
plt.show()












In [ ]:
''' Color plpotting but it's not actually useful plots. 
print("Flux W1:", fluxw1)
print("Flux W2:", fluxw2)
print("Flux W3:", fluxw3)
print("Flux W4:", fluxw4)



color_1_2 = (fluxw1 - fluxw2) / (fluxw1 +fluxw2)
color_1_3 =(fluxw1 - fluxw3) / (fluxw1 +fluxw3)
color_1_4 =(fluxw1 - fluxw4) / (fluxw1 +fluxw4)
color_2_3 =(fluxw2 - fluxw3) / (fluxw2 +fluxw3)
color_2_4 =(fluxw2 - fluxw4) / (fluxw2 +fluxw4)
color_3_4 = (fluxw3 - fluxw4) / (fluxw3 +fluxw4)

print (color_1_2)
# should be 15 color pairs
color_pairs = [(color_1_2, color_1_3, 'W1-W2', 'W1-W3'), 
                (color_1_2, color_1_4, 'W1-W2', 'W1-W4'),
                (color_1_2, color_2_3, 'W1-W2', 'W2-W3'), 
                (color_1_2, color_2_4, 'W1-W2', 'W2-W4'), 
                (color_1_2, color_3_4, 'W1-W2', 'W3-W4'), 
                (color_1_3, color_1_4, 'W1-W3', 'W1-W4'), 
                (color_1_3, color_2_3, 'W1-W3', 'W2-W3'), 
                (color_1_3, color_2_4, 'W1-W3', 'W2-W4'), 
                (color_1_3, color_3_4, 'W1-W3', 'W3-W4'), 
                (color_1_4, color_2_3, 'W1-W4', 'W2-W3'), 
                (color_1_4, color_2_4, 'W1-W4', 'W2-W4'), 
                (color_1_4, color_3_4, 'W1-W4', 'W3-W4'), 
                (color_2_3, color_2_4, 'W2-W3', 'W2-W4'), 
                (color_2_3, color_3_4, 'W2-W3', 'W3-W4'), 
                (color_2_4, color_3_4, 'W2-W4', 'W3-W4')]
    


for color_x, color_y, label_x, label_y in color_pairs:
    plt.scatter(color_x, color_y, marker = 'o')
    plt.xlabel(f'Color 1 {label_x}')
    plt.ylabel(f'Color 2 {label_y}')
    plt.title('Color-Color Diagram')
    plt.grid(True)
    plt.show()



'''

In [ ]:
#hard flux color and soft fl color. plot against luminosity. 'NET_LUM_APER_0.3-8.0 for luminosities in the xray

